# 生成器

- 只能在函数内
- 函数内任何地方只要出现了 `yield` 关键字，哪怕永远无法被执行到，函数就变成了一个生成器


通常，把含有 `yield` 的函数称之为 **生成器函数**，把调用生成器函数返回给的结果称之为 **生成器**。

但是 python 文档中把这个函数称之为 **生成器**，返回的对象称之为 **生成器迭代器**。

这里分别叫做：**生成器函数**，**生成器对象**。

生成器对象就是迭代器，因此它的运行方式和迭代器是一致的：

- 通过 `next()` 方法调用
- 每次 `next()` 方法调用都会返回生成器函数中 `yield` 表达式后面的值
- 直到生成器函数中没有 `yield` 表达式，或者 `yield` 表达式后面没有值，则抛出 `StopIteration` 异常

`yield` 关键字最根本的作用是改变了函数的性质：

1. 调用生成器函数不是直接执行其中的代码，而是返回一个对象。
2. 生成器内的代码，需要通过生成器对象来执行。

从这一点上来讲，生成器函数的作用和类差不多。

In [18]:
def my_generator():
    for i in range(10):
        yield i

print(type(my_generator))
print(type(my_generator()))


<class 'function'>
<class 'generator'>


In [19]:
import inspect

In [20]:
# 是函数，也是生成器函数
print(inspect.isfunction(my_generator))
print(inspect.isgeneratorfunction(my_generator))

True
True


In [21]:
# 生成器函数不是生成器
print(inspect.isgenerator(my_generator))
print(inspect.isgenerator(my_generator()))

False
True


# demo

In [22]:
def gen_666(meet_yield: bool = False):
    print("hello")
    if meet_yield:
        print("yield")
        yield 666
        print("back")
    print("bye")
    return "result"

In [ ]:
g1 = gen_666()
next(g1)
# return

hello
bye


StopIteration: result

In [24]:
g2 = gen_666(True)
next(g2)


hello
yield


666

In [25]:
next(g2)

back
bye


StopIteration: result

In [27]:
# 一般 yield 是搭配 for 循环使用的，但是它也可以用于生成器表达式。
def count(start=0, step=1):
    n = start
    while True:
        yield n
        n += step

for i in count(10, 2):
    print(i)
    if i > 20:
        break


10
12
14
16
18
20
22


# 生成器的4个状态

- 当调用生成器函数得到生成器对象时
    - 此时的生成器对象可以理解为初始状态
- 通过 `next()` 调用生成器对象，对应的生成器函数代码开始运行
    - 此时生成器对象进入运行状态
- 如果遇到了 `yield` 语句，`next()` 返回时
    - `yield` 语句右边的值会被作为 `next()` 的返回值
    - 生成器在 `yield` 语句处暂停，并保存当前状态，等待下一次 `next()` 调用
- 如果生成器函数运行结束，抛出 `StopIteration` 异常
    - 不管是使用了 `return` 语句显示返回值，还是默认返回 `None`，返回值都只能作为异常的值被抛出
    - 此时生成器对象进入结束状态
    - 对于已经结束的生成器对象，再调用 `next()` 会抛出 `StopIteration` 异常，且没有返回值

# 生成器的三种应用场景

- 定义一个容器类的可迭代对象，为该对象实现 `__iter__()` 方法，返回一个生成器对象。
- 定义一个处理其他可迭代对象的迭代器
- 定义一个不依赖数据存储的数据生成器

## 定义一个容器类的可迭代对象，为该对象实现 `__iter__()` 方法，返回一个生成器对象。

In [ ]:
class MyCustomIterator:
    def __init__(self, data):
        self.data = data
        self.index = -1

    def __iter__(self):
        return self

    def __next__(self):
        self.index += 1
        if self.index < self.data.size:
            return self.data.get(self.index)
        else:
            raise StopIteration

# 可迭代数据类
class MyCustonData:
    ...

    @property
    def size(self):
        return self.size

    def get(self, index):
        return self.data[index]

    def __iter__(self):
        return MyCustomIterator(self)

In [ ]:
# 使用 yield 关键字简化上述步骤
# 可迭代数据类
class MyCustonData1:
    ...

    @property
    def size(self):
        return self.size

    def get(self, index):
        return self.data[index]

    def __iter__(self):
        # index 必须是局部变量
        index = 0
        while index < self.size:
            yield self.data[index]
            index += 1

## 定义一个处理其他可迭代对象的迭代器

In [3]:
BLACK_LIST = ["白嫖", "取关"]
def shuzi_filter(actions):
    index = 0
    while index < len(actions):
        action = actions[index]
        index += 1
        if action in BLACK_LIST:
            continue
        elif "币" in action:
            yield action * 2
        else:
            yield action


actions = ["点赞", "收藏", "转发", "评论", "投币", "白嫖", "取关"]
sz_iterator = shuzi_filter(actions)
for action in sz_iterator:
    print(action)

点赞
收藏
转发
评论
投币投币


## 定义一个不依赖数据存储的数据生成器

In [4]:
def count_down(n):
    while n > 0:
        yield n
        n -= 1

for i in count_down(5):
    print(i)

5
4
3
2
1
